In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Testing").getOrCreate()


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,377 kB]
Get:13 http://archive.ubuntu.com/ubuntu ja

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, when
spark = SparkSession.builder.appName("BankChurners").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://groupfourproject.s3.ca-central-1.amazonaws.com/bank_churners.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("bank_churners.csv"), header=True, sep=',', inferSchema=True)


# Show DataFrame
df.show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [4]:
#Showing Column Type
df.printSchema()

root
 |-- CLIENTNUM: integer (nullable = true)
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: integer (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: integer (nullable = true)
 |-- Total_Relationship_Count: integer (nullable = true)
 |-- Months_Inactive_12_mon: integer (nullable = true)
 |-- Contacts_Count_12_mon: integer (nullable = true)
 |-- Credit_Limit: double (nullable = true)
 |-- Total_Revolving_Bal: integer (nullable = true)
 |-- Avg_Open_To_Buy: double (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: double (nullable = true)
 |-- Total_Trans_Amt: integer (nullable = true)
 |-- Total_Trans_Ct: integer (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: double (nullable = true)
 |-- Avg_Utilization_Ratio: double (n

In [5]:
#Dropping duplicates within the dataframe
df.dropDuplicates().show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [6]:
#Drooping NA in the dataframe
df.na.drop(how='all').show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [7]:
#Showing the head of the dataframe
df.show(10)

#Displaying the tail of the dataframe
df.tail(10)
df.show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [76]:
credit_df= df.drop('CLIENTNUM', 'Customer_Age', 'Marital_Status',
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1')

credit_df.show()

+-----------------+------+---------------+---------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|   Attrition_Flag|Gender|Dependent_count|Education_Level|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------+---------------+---------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|Existing Customer|     M|              3|    Hig

In [77]:
#Create a temporary view
credit_df.createOrReplaceTempView("bank")


**Demographic Data to Analyze**

**Gender**

In [78]:
gender = spark.sql("""
SELECT
  Attrition_Flag as Flag,
  Gender,
  COUNT(Attrition_Flag) as Count
FROM
  bank
WHERE
  Gender IN ('F', 'M')
GROUP BY
  Attrition_Flag,
  Gender
""")

gender.show()

gendercount = spark.sql("""
SELECT
  Attrition_Flag as Flag,
  COUNT(*) as Count
FROM
  bank
WHERE
  Gender IN ('F', 'M')
GROUP BY
  Attrition_Flag
""")

gendercount.show()




+-----------------+------+-----+
|             Flag|Gender|Count|
+-----------------+------+-----+
|Existing Customer|     F| 4428|
|Attrited Customer|     F|  930|
|Existing Customer|     M| 4072|
|Attrited Customer|     M|  697|
+-----------------+------+-----+

+-----------------+-----+
|             Flag|Count|
+-----------------+-----+
|Existing Customer| 8500|
|Attrited Customer| 1627|
+-----------------+-----+



**Dependent Count**

In [79]:
dependent_count= spark.sql("""
SELECT
  Attrition_Flag as Flag,
  COUNT(*) as DependentCount,
  CASE
    WHEN Dependent_count BETWEEN 0 AND 5 THEN '0-5'
    ELSE 'Other'
  END AS DependentRange
FROM
  bank
GROUP BY
  Attrition_Flag,
  CASE
    WHEN Dependent_count BETWEEN 0 AND 5 THEN '0-5'
    ELSE 'Other'
  END
ORDER BY
  DependentRange
""")
dependent_count.show()

+-----------------+--------------+--------------+
|             Flag|DependentCount|DependentRange|
+-----------------+--------------+--------------+
|Attrited Customer|          1627|           0-5|
|Existing Customer|          8500|           0-5|
+-----------------+--------------+--------------+



**Education**

In [80]:
education = spark.sql("""
SELECT
  Attrition_Flag as Flag,
  Education_Level as Education,
  COUNT(Education_Level) as EducationCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Education_Level
ORDER BY
  EducationCount
""")
education.show()

+-----------------+-------------+--------------+
|             Flag|    Education|EducationCount|
+-----------------+-------------+--------------+
|Attrited Customer|Post-Graduate|            92|
|Attrited Customer|    Doctorate|            95|
|Attrited Customer|      College|           154|
|Attrited Customer|   Uneducated|           237|
|Attrited Customer|      Unknown|           256|
|Attrited Customer|  High School|           306|
|Existing Customer|    Doctorate|           356|
|Existing Customer|Post-Graduate|           424|
|Attrited Customer|     Graduate|           487|
|Existing Customer|      College|           859|
|Existing Customer|   Uneducated|          1250|
|Existing Customer|      Unknown|          1263|
|Existing Customer|  High School|          1707|
|Existing Customer|     Graduate|          2641|
+-----------------+-------------+--------------+



**Income Category**

In [82]:
income = spark.sql("""
SELECT
  Attrition_Flag as Flag,
  Income_Category as Income,
  COUNT(*) as IncomeCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Income_Category
ORDER BY
  IncomeCount
""")

income.show()

+-----------------+--------------+-----------+
|             Flag|        Income|IncomeCount|
+-----------------+--------------+-----------+
|Attrited Customer|       $120K +|        126|
|Attrited Customer|       Unknown|        187|
|Attrited Customer|   $60K - $80K|        189|
|Attrited Customer|  $80K - $120K|        242|
|Attrited Customer|   $40K - $60K|        271|
|Existing Customer|       $120K +|        601|
|Attrited Customer|Less than $40K|        612|
|Existing Customer|       Unknown|        925|
|Existing Customer|   $60K - $80K|       1213|
|Existing Customer|  $80K - $120K|       1293|
|Existing Customer|   $40K - $60K|       1519|
|Existing Customer|Less than $40K|       2949|
+-----------------+--------------+-----------+



**Behavioral (Bank) Data**

**Months on book**

In [14]:
month_book = spark.sql("""SELECT
  Attrition_Flag as Flag,
  Months_on_book as Months,
  COUNT(*) as MonthsCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Months_on_book
ORDER BY
  Months
""")
month_book.show()

+-----------------+------+-----------+
|             Flag|Months|MonthsCount|
+-----------------+------+-----------+
|Existing Customer|    13|         63|
|Attrited Customer|    13|          7|
|Existing Customer|    14|         15|
|Attrited Customer|    14|          1|
|Existing Customer|    15|         25|
|Attrited Customer|    15|          9|
|Existing Customer|    16|         26|
|Attrited Customer|    16|          3|
|Attrited Customer|    17|          4|
|Existing Customer|    17|         35|
|Attrited Customer|    18|         13|
|Existing Customer|    18|         45|
|Attrited Customer|    19|          6|
|Existing Customer|    19|         57|
|Attrited Customer|    20|         13|
|Existing Customer|    20|         61|
|Attrited Customer|    21|         10|
|Existing Customer|    21|         73|
|Attrited Customer|    22|         20|
|Existing Customer|    22|         85|
+-----------------+------+-----------+
only showing top 20 rows



**Total Relationsip Count**

In [15]:
total_relationship = spark.sql("""SELECT
  Attrition_Flag as Flag,
  Total_Relationship_Count as TotalRelationship,
  COUNT(*) as MonthsCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Total_Relationship_Count
ORDER BY
  TotalRelationship
""")
total_relationship.show()

+-----------------+-----------------+-----------+
|             Flag|TotalRelationship|MonthsCount|
+-----------------+-----------------+-----------+
|Existing Customer|                1|        677|
|Attrited Customer|                1|        233|
|Existing Customer|                2|        897|
|Attrited Customer|                2|        346|
|Attrited Customer|                3|        400|
|Existing Customer|                3|       1905|
|Attrited Customer|                4|        225|
|Existing Customer|                4|       1687|
|Existing Customer|                5|       1664|
|Attrited Customer|                5|        227|
|Existing Customer|                6|       1670|
|Attrited Customer|                6|        196|
+-----------------+-----------------+-----------+



**Months Inactive 12 months**

In [16]:
months_inactive = spark.sql("""SELECT
  Attrition_Flag as Flag,
  Months_Inactive_12_mon as MonthsInactive,
  COUNT(*) as InactiveCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Months_Inactive_12_mon
ORDER BY
  MonthsInactive
""")
months_inactive.show()

+-----------------+--------------+-------------+
|             Flag|MonthsInactive|InactiveCount|
+-----------------+--------------+-------------+
|Existing Customer|             0|           14|
|Attrited Customer|             0|           15|
|Existing Customer|             1|         2133|
|Attrited Customer|             1|          100|
|Existing Customer|             2|         2777|
|Attrited Customer|             2|          505|
|Attrited Customer|             3|          826|
|Existing Customer|             3|         3020|
|Attrited Customer|             4|          130|
|Existing Customer|             4|          305|
|Existing Customer|             5|          146|
|Attrited Customer|             5|           32|
|Existing Customer|             6|          105|
|Attrited Customer|             6|           19|
+-----------------+--------------+-------------+



**Contacts Count 12 months**

In [17]:
contacts = spark.sql("""SELECT
  Attrition_Flag as Flag,
  Contacts_Count_12_mon as Contact,
  COUNT(*) as MonthsCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Contacts_Count_12_mon
ORDER BY
  Contact
""")
contacts.show()

+-----------------+-------+-----------+
|             Flag|Contact|MonthsCount|
+-----------------+-------+-----------+
|Existing Customer|      0|        392|
|Attrited Customer|      0|          7|
|Existing Customer|      1|       1391|
|Attrited Customer|      1|        108|
|Existing Customer|      2|       2824|
|Attrited Customer|      2|        403|
|Attrited Customer|      3|        681|
|Existing Customer|      3|       2699|
|Attrited Customer|      4|        315|
|Existing Customer|      4|       1077|
|Existing Customer|      5|        117|
|Attrited Customer|      5|         59|
|Attrited Customer|      6|         54|
+-----------------+-------+-----------+



**Credit Limit**

In [18]:
credit = spark.sql("""SELECT
  Attrition_Flag as Flag,
  Credit_Limit as CreditLimit,
  COUNT(*) as CreditCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Credit_Limit
ORDER BY
  CreditCount
""")
credit.show()

+-----------------+-----------+-----------+
|             Flag|CreditLimit|CreditCount|
+-----------------+-----------+-----------+
|Existing Customer|    23665.0|          1|
|Existing Customer|    12932.0|          1|
|Attrited Customer|     1491.0|          1|
|Existing Customer|    14035.0|          1|
|Attrited Customer|    11422.0|          1|
|Existing Customer|     4844.0|          1|
|Existing Customer|     4135.0|          1|
|Existing Customer|     5478.0|          1|
|Attrited Customer|     2066.0|          1|
|Existing Customer|     1525.0|          1|
|Existing Customer|     6966.0|          1|
|Existing Customer|    19999.0|          1|
|Existing Customer|    18379.0|          1|
|Existing Customer|     5443.0|          1|
|Existing Customer|     7264.0|          1|
|Existing Customer|     7112.0|          1|
|Existing Customer|     1671.0|          1|
|Existing Customer|     5585.0|          1|
|Attrited Customer|     5557.0|          1|
|Existing Customer|    25666.0| 

**Total Trans Amt**

In [19]:
total_amt = spark.sql("""SELECT
  Attrition_Flag as Flag,
  Total_Trans_Amt as TotalTrans,
  COUNT(*) as TransCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Total_Trans_Amt
ORDER BY
  TotalTrans
""")
total_amt.show()

+-----------------+----------+----------+
|             Flag|TotalTrans|TransCount|
+-----------------+----------+----------+
|Attrited Customer|       510|         1|
|Attrited Customer|       530|         1|
|Attrited Customer|       563|         1|
|Attrited Customer|       569|         1|
|Attrited Customer|       594|         1|
|Attrited Customer|       596|         1|
|Attrited Customer|       597|         1|
|Attrited Customer|       602|         1|
|Attrited Customer|       615|         1|
|Attrited Customer|       643|         1|
|Attrited Customer|       644|         1|
|Attrited Customer|       646|         1|
|Attrited Customer|       647|         2|
|Attrited Customer|       654|         1|
|Attrited Customer|       660|         1|
|Attrited Customer|       673|         1|
|Attrited Customer|       678|         1|
|Attrited Customer|       683|         1|
|Attrited Customer|       687|         1|
|Attrited Customer|       689|         1|
+-----------------+----------+----

**Total Trans Count**

In [20]:
total_count = spark.sql("""SELECT
  Attrition_Flag as Flag,
  Total_Trans_Ct as TotalTransCount,
  COUNT(*) as CountTotalTrans
FROM
  bank
GROUP BY
  Attrition_Flag,
  Total_Trans_Ct
ORDER BY
  TotalTransCount
""")
total_count.show()


+-----------------+---------------+---------------+
|             Flag|TotalTransCount|CountTotalTrans|
+-----------------+---------------+---------------+
|Attrited Customer|             10|              4|
|Attrited Customer|             11|              1|
|Existing Customer|             11|              1|
|Attrited Customer|             12|              4|
|Existing Customer|             13|              2|
|Attrited Customer|             13|              3|
|Existing Customer|             14|              1|
|Attrited Customer|             14|              8|
|Existing Customer|             15|              4|
|Attrited Customer|             15|             12|
|Existing Customer|             16|              5|
|Attrited Customer|             16|              8|
|Attrited Customer|             17|             10|
|Existing Customer|             17|              3|
|Attrited Customer|             18|             15|
|Existing Customer|             18|              8|
|Attrited Cu

In [21]:
from pyspark.sql import functions as F
from pyspark.sql import types as T


#Gives a label to each category
#creates onehot encoding vector
from pyspark.ml.feature import StringIndexer, OneHotEncoder

#Used to creat vector from features and modeling takes vector as an input
from pyspark.ml.feature import VectorAssembler

#Used for classification problems
from pyspark.ml.classification import DecisionTreeClassifier


In [88]:
attrition_df = credit_df.withColumn("Attrition_Flag", F.when(F.col("Attrition_Flag") =='Attrited Customer', 0).otherwise(1))
card_df = credit_df.withColumn(
    "Card_Category",
    F.when(F.col("Card_Category") == 'Blue', 0)
     .when(F.col("Card_Category") == 'Gold', 1)
     .when(F.col("Card_Category") == 'Platinum', 3)
     .when(F.col("Card_Category") == 'Silver', 4)
     .otherwise(F.col("Card_Category"))  # handle unexpected values
)
attrition_df.show(3, truncate=False)

attrition_df.groupby("Attrition_Flag").count().show()
card_df.groupby("Card_Category").count().show()


+--------------+------+---------------+---------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|Attrition_Flag|Gender|Dependent_count|Education_Level|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+--------------+------+---------------+---------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|1             |M     |3              |High School    |$60

**1st Modeling**

In [93]:
(train_df, test_df) = attrition_df.randomSplit([0.8, 0.2], 11)
(train_df, test_df) = card_df.randomSplit([0.8, 0.2], 11)
print("Number of train samples: " + str(train_df.count()))
print("Number of test samples: " + str(test_df.count()))

Number of train samples: 8102
Number of test samples: 2025


In [95]:
# Apply StringIndexer to the Attrition_Flag column
attrition_indexer = StringIndexer(inputCol="Attrition_Flag", outputCol="Attrition_Flag_Index")
indexed_df = attrition_indexer.fit(credit_df).transform(credit_df)

# Apply StringIndexer to the Card_Category column
card_indexer = StringIndexer(inputCol="Card_Category", outputCol="Card_Category_Index")
indexed_df = card_indexer.fit(indexed_df).transform(indexed_df)

# Show the transformed DataFrame
indexed_df.show(truncate=False)

+-----------------+------+---------------+---------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+--------------------+-------------------+
|Attrition_Flag   |Gender|Dependent_count|Education_Level|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Attrition_Flag_Index|Card_Category_Index|
+-----------------+------+---------------+---------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+----------

In [96]:
inputCols=[
 'Dependent_count',
 'Months_on_book',
 'Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Avg_Open_To_Buy',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio'
    ]

outputCol="features"

vector_assembler = VectorAssembler(inputCols= inputCols, outputCol=outputCol )
train_df = vector_assembler.transform(train_df)

In [97]:
train_df.show(3, truncate=False)

+-----------------+------+---------------+---------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+-------------------------------------------------------------------------+
|Attrition_Flag   |Gender|Dependent_count|Education_Level|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|features                                                                 |
+-----------------+------+---------------+---------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+------------

In [98]:
modeling_df = train_df.select(['features', 'Attrition_Flag'])
modeling_df.show()

modeling_df.tail(10)
modeling_df.show()

+--------------------+-----------------+
|            features|   Attrition_Flag|
+--------------------+-----------------+
|[0.0,30.0,5.0,2.0...|Attrited Customer|
|[0.0,19.0,2.0,1.0...|Attrited Customer|
|[0.0,40.0,5.0,4.0...|Attrited Customer|
|[0.0,56.0,3.0,4.0...|Attrited Customer|
|[0.0,15.0,3.0,1.0...|Attrited Customer|
|[0.0,36.0,4.0,3.0...|Attrited Customer|
|[0.0,36.0,4.0,3.0...|Attrited Customer|
|[0.0,37.0,5.0,2.0...|Attrited Customer|
|[0.0,38.0,4.0,3.0...|Attrited Customer|
|[0.0,40.0,3.0,4.0...|Attrited Customer|
|[0.0,56.0,5.0,4.0...|Attrited Customer|
|[0.0,20.0,5.0,1.0...|Attrited Customer|
|[0.0,21.0,4.0,2.0...|Attrited Customer|
|[0.0,25.0,5.0,3.0...|Attrited Customer|
|[0.0,36.0,1.0,3.0...|Attrited Customer|
|[0.0,36.0,2.0,3.0...|Attrited Customer|
|[0.0,36.0,2.0,3.0...|Attrited Customer|
|[0.0,36.0,4.0,3.0...|Attrited Customer|
|[0.0,36.0,5.0,3.0...|Attrited Customer|
|[0.0,38.0,2.0,3.0...|Attrited Customer|
+--------------------+-----------------+
only showing top

In [29]:
#Creating a DecisionTreeClassifier model
model_credit = DecisionTreeClassifier(labelCol="Attrition_Flag", featuresCol="features")

#Train model
model_credit = model_credit.fit(modeling_df)

In [30]:
predicton = model_credit.transform(modeling_df)
predicton.show()

+--------------------+--------------+-------------+--------------------+----------+
|            features|Attrition_Flag|rawPrediction|         probability|prediction|
+--------------------+--------------+-------------+--------------------+----------+
|[0.0,30.0,5.0,2.0...|             0|[133.0,113.0]|[0.54065040650406...|       0.0|
|[0.0,19.0,2.0,1.0...|             0| [125.0,19.0]|[0.86805555555555...|       0.0|
|[0.0,40.0,5.0,4.0...|             0|[133.0,113.0]|[0.54065040650406...|       0.0|
|[0.0,56.0,3.0,4.0...|             0| [14.0,137.0]|[0.09271523178807...|       1.0|
|[0.0,15.0,3.0,1.0...|             0|   [18.0,3.0]|[0.85714285714285...|       0.0|
|[0.0,36.0,4.0,3.0...|             0| [531.0,79.0]|[0.87049180327868...|       0.0|
|[0.0,36.0,4.0,3.0...|             0|[30.0,1008.0]|[0.02890173410404...|       1.0|
|[0.0,37.0,5.0,2.0...|             0| [531.0,79.0]|[0.87049180327868...|       0.0|
|[0.0,38.0,4.0,3.0...|             0|[133.0,113.0]|[0.54065040650406...|    

In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Attrition_Flag")
area = evaluator.evaluate(predicton)

print(area)

0.9354346039699568


**1st Test Predictions**

In [32]:
#Test data- transform test data using all tranformers and estimators

test_df = attrition_indexer.transform(test_df)
test_df = vector_assembler.transform(test_df)
test_predication = model_credit.transform(test_df)

test_predication.show()

+--------------+------+---------------+---------------+---------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+-------------+--------------------+-------------+--------------------+----------+
|Attrition_Flag|Gender|Dependent_count|Education_Level|Income_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|AttritedIndex|            features|rawPrediction|         probability|prediction|
+--------------+------+---------------+---------------+---------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+------------

In [33]:
area = evaluator.evaluate(test_predication)

print(area)

0.9237926606284816


**2nd Modeling**

In [64]:
from pyspark.sql import functions as F

# Perform groupBy and count operation on 'Attrition_Flag' column
attrition_flag_counts = credit_df.groupBy("Attrition_Flag").count()

# Join the original DataFrame with the DataFrame containing counts
updated_df = credit_df.join(attrition_flag_counts, "Attrition_Flag", "left")

df_8500 = updated_df.filter(updated_df["count"]==8500)
df_sample =df_8500.sample(fraction=1627.0/8500.0, seed=1)

df_1627 = updated_df.filter(updated_df["count"]==1627)
updated_df = df_sample.union(df_1627)

# Show the updated DataFrame
updated_df.show()



+-----------------+------+---------------+---------------+---------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+-----+
|   Attrition_Flag|Gender|Dependent_count|Education_Level|Income_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|count|
+-----------------+------+---------------+---------------+---------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+-----+
|Existing Customer|     M|              3|       Graduate|   $80K - $120K

In [65]:
# Define the new value
new_value = 1627

# Use `withColumn` and `when` function to create a new column with updated values
new_df = updated_df.withColumn("count", when(updated_df["count"] == 8500, new_value).otherwise(updated_df["count"]))

# Show the updated DataFrame
new_df.show()

+-----------------+------+---------------+---------------+---------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+-----+
|   Attrition_Flag|Gender|Dependent_count|Education_Level|Income_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|count|
+-----------------+------+---------------+---------------+---------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+-----+
|Existing Customer|     M|              3|       Graduate|   $80K - $120K

In [66]:
new_attrited_df = new_df.withColumn("Attrition_Flag", F.when(F.col("Attrition_Flag") =='Attrited Customer', 0).otherwise(1))
new_attrited_df.show(3, truncate=False)

new_attrited_df.groupby("Attrition_Flag").count().show()

+--------------+------+---------------+---------------+---------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+-----+
|Attrition_Flag|Gender|Dependent_count|Education_Level|Income_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|count|
+--------------+------+---------------+---------------+---------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+-----+
|1             |M     |3              |Graduate       |$80K - $120K   |36         

In [67]:
(train_df, test_df) = new_attrited_df.randomSplit([0.8, 0.2], 11)
print("Number of train samples: " + str(train_df.count()))
print("Number of test samples: " + str(test_df.count()))

Number of train samples: 2632
Number of test samples: 679


IllegalArgumentException: requirement failed: Output column AttritedIndex already exists.

In [69]:
inputCols=[
 'Dependent_count',
 'Months_on_book',
 'Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Avg_Open_To_Buy',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio'
    ]

outputCol="features"

vector_assembler2 = VectorAssembler(inputCols= inputCols, outputCol=outputCol )
train_df2 = vector_assembler2.transform(train_df)

In [70]:
modeling_df = train_df.select(['features', 'Attrition_Flag'])
modeling_df.show()

modeling_df.tail(10)
modeling_df.show()

+--------------------+--------------+
|            features|Attrition_Flag|
+--------------------+--------------+
|[0.0,13.0,2.0,3.0...|             1|
|[0.0,46.0,6.0,2.0...|             1|
|[0.0,53.0,3.0,2.0...|             1|
|[0.0,30.0,3.0,2.0...|             1|
|[0.0,36.0,1.0,2.0...|             1|
|[0.0,56.0,5.0,2.0...|             1|
|[0.0,36.0,1.0,1.0...|             1|
|[0.0,51.0,3.0,3.0...|             1|
|[0.0,36.0,3.0,1.0...|             1|
|[0.0,45.0,5.0,2.0...|             1|
|[0.0,45.0,6.0,3.0...|             1|
|[0.0,56.0,4.0,2.0...|             1|
|[0.0,56.0,4.0,3.0...|             1|
|[0.0,17.0,4.0,1.0...|             1|
|[0.0,20.0,5.0,1.0...|             1|
|[0.0,26.0,3.0,1.0...|             1|
|[0.0,26.0,6.0,1.0...|             1|
|[0.0,32.0,2.0,2.0...|             1|
|[0.0,36.0,2.0,2.0...|             1|
|[0.0,36.0,4.0,3.0...|             1|
+--------------------+--------------+
only showing top 20 rows

+--------------------+--------------+
|            features|At

In [71]:
#Creatind a DecisionTreeClassifier model
model_credit = DecisionTreeClassifier(labelCol="Attrition_Flag", featuresCol="features")

#Train model
model_credit = model_credit.fit(modeling_df)

In [72]:
predicton = model_credit.transform(modeling_df)
predicton.show()

+--------------------+--------------+-------------+--------------------+----------+
|            features|Attrition_Flag|rawPrediction|         probability|prediction|
+--------------------+--------------+-------------+--------------------+----------+
|[0.0,13.0,2.0,3.0...|             1|  [4.0,612.0]|[0.00649350649350...|       1.0|
|[0.0,46.0,6.0,2.0...|             1|  [4.0,612.0]|[0.00649350649350...|       1.0|
|[0.0,53.0,3.0,2.0...|             1|  [4.0,612.0]|[0.00649350649350...|       1.0|
|[0.0,30.0,3.0,2.0...|             1|   [5.0,74.0]|[0.06329113924050...|       1.0|
|[0.0,36.0,1.0,2.0...|             1| [10.0,194.0]|[0.04901960784313...|       1.0|
|[0.0,56.0,5.0,2.0...|             1|  [4.0,612.0]|[0.00649350649350...|       1.0|
|[0.0,36.0,1.0,1.0...|             1| [10.0,194.0]|[0.04901960784313...|       1.0|
|[0.0,51.0,3.0,3.0...|             1| [56.0,244.0]|[0.18666666666666...|       1.0|
|[0.0,36.0,3.0,1.0...|             1|   [25.0,2.0]|[0.92592592592592...|    

In [73]:

evaluator = MulticlassClassificationEvaluator(labelCol="Attrition_Flag")
area = evaluator.evaluate(predicton)

print(area)

0.9201211456673946
